# 데이터 전처리

In [244]:
# 결측치 체크 및 처리
# 중복 체크 및 처리
# 이상치 체크 및 처리
# 데이터 타입 정리
# 데이터 타입 변환
# 데이터 인코딩 처리
# 데이터 스케일링 처리

In [245]:
# 모듈 로딩
import pandas as pd
from numpy import nan

In [246]:
# 데이터 생성
names = ['a', 'b', 'c', 'd', 'e',
        'f', 'g', 'h', 'i', 'j']
ages = [0, 4, 21, 20, 38, 100, 40, 61, 11, 23]
genders = ['M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M']

In [247]:
peopleDF = pd.DataFrame({'Name':names, 'Age':ages, 'Gender':genders})

In [248]:
peopleDF.head(3)

,Name,Age,Gender
0,a,0,M
1,b,4,F
2,c,21,M


In [249]:
peopleDF.dtypes

Name      object
Age        int64
Gender    object
dtype: object

# 데이터 전처리 - (1) 데이터 타입 체크 및 처리

In [250]:
# astype('데이터 타입'): 반드시 원본 저장 -> 적용 / inplace 매개 변수 X

In [251]:
# 성별 컬럼 object -> category
peopleDF.Gender.astype('category')

0    M
1    F
2    M
3    M
4    F
5    M
6    M
7    M
8    M
9    M
Name: Gender, dtype: category
Categories (2, object): ['F', 'M']

In [252]:
# 연속형 데이터 -> 범주형 데이터 변환
# *astype('데이터 타입') -> 대량 범주형 생성 문제
# *cut(데이터, 구간 정보, 구간 라벨) -> 범위 즉 구간 설정 -> 범주 변경

In [253]:
peopleDF.Age
# 아동, 청년, 중년, 장년, 노년

0      0
1      4
2     21
3     20
4     38
5    100
6     40
7     61
8     11
9     23
Name: Age, dtype: int64

In [254]:
ret1, bins = pd.cut(peopleDF.Age, bins = 5, labels = ['아동', '청년', '중년', '장년', '노년'],
              right = False, retbins = True, include_lowest = True)

In [255]:
ret1

0    아동
1    아동
2    청년
3    청년
4    청년
5    노년
6    중년
7    장년
8    아동
9    청년
Name: Age, dtype: category
Categories (5, object): ['아동' < '청년' < '중년' < '장년' < '노년']

In [256]:
bins

array([  0. ,  20. ,  40. ,  60. ,  80. , 100.1])

In [257]:
peopleDF['Age_Ctg'] = ret1

In [258]:
peopleDF.head(3)

,Name,Age,Gender,Age_Ctg
0,a,0,M,아동
1,b,4,F,아동
2,c,21,M,청년


In [259]:
# One Hot Encoding 변환

In [260]:
ret2 = pd.get_dummies(peopleDF.Age_Ctg)
ret2

,아동,청년,중년,장년,노년
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
5,0,0,0,0,1
6,0,0,1,0,0
7,0,0,0,1,0
8,1,0,0,0,0
9,0,1,0,0,0


In [261]:
p2 = pd.concat([peopleDF, ret2], axis = 1)

In [262]:
p2.head()

,Name,Age,Gender,Age_Ctg,아동,청년,중년,장년,노년
0,a,0,M,아동,1,0,0,0,0
1,b,4,F,아동,1,0,0,0,0
2,c,21,M,청년,0,1,0,0,0
3,d,20,M,청년,0,1,0,0,0
4,e,38,F,청년,0,1,0,0,0


In [263]:
sel = p2.loc[0, '아동':]

In [264]:
sel.to_list()

[1, 0, 0, 0, 0]

In [265]:
sel.to_numpy()

array([1, 0, 0, 0, 0], dtype=object)

# scikit-learn 사용 문자열 범주형 인코딩

In [266]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

In [267]:
# Encoder 객체 생성
le = LabelEncoder()

In [268]:
le.fit(peopleDF.Age_Ctg)

LabelEncoder()

In [269]:
le.classes_

array(['노년', '아동', '장년', '중년', '청년'], dtype=object)

In [270]:
# 정수 인코딩
en_age = le.transform(peopleDF.Age_Ctg)

In [271]:
# 정수 -> 문자열 돌리기
le.inverse_transform(en_age)

array(['아동', '아동', '청년', '청년', '청년', '노년', '중년', '장년', '아동', '청년'],
      dtype=object)

In [272]:
ohe = OneHotEncoder(sparse = False)

In [273]:
ohe.fit(peopleDF.Age_Ctg.to_numpy().reshape(-1, 1))

OneHotEncoder(sparse=False)

In [274]:
ohe.transform(peopleDF.Age_Ctg.to_numpy().reshape(-1, 1))

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [275]:
import seaborn as sns

In [276]:
titanicDF = sns.load_dataset('titanic')

In [277]:
titanicDF

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True
